<a href="https://colab.research.google.com/github/MichaelZhao21/stickerinator/blob/master/stickerinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Margins Function

This will take in an image and add a 5% margin based on the most common color in the 4 corners (the background color). The inputs of this block can be any image type and will output images with `exp-` appended to the name.

In [6]:
import math
from PIL import Image, ImageFilter
import numpy as np

# Create output dir
!mkdir margins

def add_margin(filename):
    # Read in the image file
    im = Image.open(filename).convert('RGB')
    ima = np.array(im)

    # Apply a gaussian blur using 1% of the pixels (this number must be odd w/ min of 1)
    blur_px = max(round(min(im.size[0], im.size[1]) * 0.01), 1);
    imb = im.filter(ImageFilter.GaussianBlur(radius=2))
    imba = np.array(imb)

    # Show images
    print('Original image dims:', im.size[1], 'x', im.size[0])
    print('Blurred with', blur_px, 'x', blur_px, 'Gaussian filter')

    # Utility function to round to the 10s place
    round_tens = np.vectorize(lambda x: round(x / 10) * 10)
    ceil_tens = np.vectorize(lambda x: min(math.ceil(x / 10) * 10, 250))

    # Calculate how many pixels to get (~5% of the length of the shortest side)
    width = im.size[0]
    height = im.size[1]
    px = round(min(width, height) * 0.05)
    csz = px * px

    # Get corners, where corners are stored in clockwise order starting from the top left
    corners = np.zeros((4, px, px, 3))
    corners[0] = imba[:px, :px]
    corners[1] = imba[:px, -px:]
    corners[2] = imba[-px:, :px]
    corners[3] = imba[-px:, -px:]

    # We first want to store all counts of color values in a dictionary
    # Rounding the values to the tens place allows for us to correct for slight changes in color
    # If there is a color that has a majority in the dictionary, it is probably the background color
    corner_list = np.reshape(corners, (px * px * 4, 3))
    rounded_corner_list = ceil_tens(corner_list)
    corner_dict = {}
    for item in rounded_corner_list:
        item_t = tuple(item)
        if item_t in corner_dict.keys():
            corner_dict[item_t] = corner_dict[item_t] + 1
        else:
            corner_dict[item_t] = 1

    # Sort the dictionary to get the most common color among all these pixels
    corner_dict_sorted = dict(
        sorted(corner_dict.items(), key=lambda item: -item[1]))
    color_list = list(corner_dict_sorted.keys())
    bg = color_list[0]

    color_img = np.empty((40, 40, 3), np.uint8)
    color_img[:, :] = bg

    clist_img = np.empty((20, 20 * len(color_list), 3), np.uint8)
    for i, val in enumerate(color_list):
        clist_img[:, (i*20):((i+1)*20)] = val

    print('Approx corner color counts:', corner_dict_sorted)
    display(Image.fromarray(clist_img))
    print('\nMost common (probably background) color:', bg)
    display(Image.fromarray(color_img))

    # Expand the edges of the image by 10% of the longest edge with the background
    # color as the fill color.
    mg = round(max(width, height) * 0.1)
    exp_img = np.full((height+(2*mg), width+(2*mg), 3), bg, np.float32)
    exp_img[mg:(height+mg),mg:(width+mg)] = ima
    ei = Image.fromarray((exp_img).astype(np.uint8))

    print('\nAdded', mg, 'pixel margin to edges:')
    display(ei)

    # Save
    fn_no_pre = re.sub('^.*\/', '', filename)
    ei.save(f'margins/exp-{fn_no_pre}')

mkdir: cannot create directory ‘margins’: File exists


# Remove Background, Add Border, and Crop (Process Image) Function

In [14]:
from PIL import Image, ImageDraw
import numpy as np
import re

# Create output dir
!mkdir processed

def process_img(filename, add_border, crop, threshold=150):
    # Read in the image file
    im = Image.open(filename).convert('RGB')

    # Floodfill op
    print('\nCreating image mask...')
    ImageDraw.floodfill(im, xy=(10, 10), value=(255, 0, 255), thresh=threshold)

    # Mask
    am = np.array(im)
    am[(am[:, :, 0:3] != [255, 0, 255]).any(2)] = [0, 255, 0]
    mask = Image.fromarray(am)
    print('Finished image mask!')
    display(mask)

    if add_border:
        # Border checking function
        maa = [255, 0, 255]
        is_border = lambda r, c: (am[r, c] != maa).any() and ((am[r-1, c] == maa).all() or (am[r, c+1] == maa).all() or (am[r+1, c] == maa).all() or (am[r, c-1] == maa).all())

        # Find and draw borders on mask
        da = np.zeros(am.shape)
        draw = ImageDraw.Draw(mask)
        RAD = 8 # Border radius
        print('\nAdding borders on mask... Done with row ', end='')
        for r in range(1, am.shape[0] - 1):
            for c in range(1, am.shape[1] - 1):
                if is_border(r, c):
                    draw.ellipse((c-RAD,r-RAD,c+RAD,r+RAD), (0,255,0), None)
            print(r, end=' | ')
        print('\nFinished adding borders on mask!')

    # Overlay the adjusted mask onto the original mask if borders were added
    if add_border:
        # Add the 2 masks together
        print('\nCombining masks...')
        ma = np.array(mask)
        ma = ma + am

        # Fix overflowed color values to 255
        fix = lambda x : 255 if x > 0 else 0
        fixv = np.vectorize(fix)
        bma = fixv(ma)

        # Overwrite bm and display
        mask = Image.fromarray(bma.astype(np.uint8))
        print('Finished image overlay with borders!')
        display(mask)

    # Apply mask to original image
    print('\nApplying mask to original image...')
    im.putalpha(255)
    imm = np.array(im)
    bma = np.array(mask)
    imm[(bma[:, :, 0:3] == [255, 255, 255]).all(2)] = [255, 255, 255, 255]
    imm[(bma[:, :, 0:3] == [255, 0, 255]).all(2)] = [0, 0, 0, 0]
    print('Finished masking original image!')
    fim = Image.fromarray(imm)

    # Crop the image
    if crop:
        # Get the max/min x/y of the image
        maa = [255, 0, 255]
        maxh, maxw, minh, minw = 0, 0, am.shape[0], am.shape[1]
        print('\nGetting image bounds...')
        for r in range(1, am.shape[0] - 1):
            for c in range(1, am.shape[1] - 1):
                if (bma[r, c] != maa).any():
                    if maxh < r:
                        maxh = r
                    if minh > r:
                        minh = r
                    if maxw < c:
                        maxw = c
                    if minw > c:
                        minw = c
        
        # Add 5% margin to the base crop
        mg = round(max(maxw - minw, maxh - minh) * 0.05)
        print(f'Image bounds are (x, y): ({minw}, {minh}) to ({maxw}, {maxh})')
        print('Cropping to bounds with', mg, 'px margin...')
        minw = max(0, minw - mg)
        minh = max(0, minh - mg)
        maxw = min(am.shape[1], maxw + mg)
        maxh = min(am.shape[0], maxh + mg)

        # Actually crop the image
        print('Cropping bounds (x, y, dx, dy):', (minw, minh, maxw - minw, maxh - minh))
        fim = fim.crop((minw, minh, maxw, maxh))
        print('Finished cropping image! Cropped size:', fim.size)
    
    # Save final image
    display(fim)
    fn_no_ext = re.sub('\..*$', '', filename)
    fn_no_ext_pre = re.sub('^.*/', '', fn_no_ext)
    fim.save(f'processed/done-{fn_no_ext_pre}.png')

mkdir: cannot create directory ‘processed’: File exists


# Runner

Make sure to initialize the functions above before running!

In [ ]:
# Input file name
FILE_NAME = 'file.jpg'

# Add a margin to the input file (comment out if not needed!)
add_margin(FILE_NAME)
FILE_NAME = f'margins/exp-{FILE_NAME}'

# Process the image
# The last 3 parameters are options that can be changed
process_img(FILE_NAME, add_border=True, crop=True, threshold=50)

# Runner for Folder (Add Margin)

This will iterate through all images in a folder and process them! Make sure you only have **one** layer to your folder or it will crash :(

In [ ]:
import os

FOLDER_NAME = 'needs-margins'

for root, subdirs, files in os.walk(FOLDER_NAME):
    for filename in files:
        full_filename = os.path.join(root, filename)
        add_margin(full_filename)

# Runner for Folders (Process Image)

In [ ]:
import os

FOLDER_NAME = 'has-margins'

for root, subdirs, files in os.walk(FOLDER_NAME):
    for filename in files:
        full_filename = os.path.join(root, filename)
        process_img(full_filename, add_border=True, crop=True, threshold=150)

# Download a Folder!

Hacky workaround to download a folder from google colab (essentially just zips it with the command `zip -r <output-zip-name>.zip <input-folder-path>`

In [ ]:
!zip -r processed.zip processed